In [7]:
from gensim import *
import spacy
import gensim.downloader

In [5]:
for model_name in list(gensim.downloader.info()['models'].keys()):
    print(model_name)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [7]:
g_news = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Paris está para França, aassim como Londres está para ... (Inglaterra)

# (Paris - França) + Italia= Roma (positive =['Paris', 'Italia'], negative =['França'])

In [11]:
nlp = spacy.load('en_core_web_sm')

In [40]:
frase  = "Happy families are all alike; every unhappy family is unhappy in its own Way, Alessandro Di Lauro"

In [41]:
text = nlp(frase)

In [42]:
txt = [token.lemma_ for token in text if not token.is_stop and not token.is_punct]

In [43]:
print(txt)

['happy', 'family', 'alike', 'unhappy', 'family', 'unhappy', 'way', 'Alessandro', 'Di', 'Lauro']


In [44]:
txt2 = [token.tag_ for token in text]

print(txt2[0])

JJ


In [83]:
capital = g_news.most_similar(['paris', 'rome'], ['france'])

In [84]:
print(capital)

[('joel', 0.48578494787216187), ('dubai', 0.4609808027744293), ('oliver', 0.4578564763069153), ('2Bdrm_suite', 0.45590725541114807), ('heidi', 0.4545368254184723), ('samuel', 0.4514700770378113), ('jh', 0.4510999917984009), ('florence', 0.44964924454689026), ('lohan', 0.44291403889656067), ('jessie', 0.44002825021743774)]


In [71]:
w = g_news.most_similar('Porsche', topn=4)

In [72]:
print(w)

[('Audi', 0.7602351307868958), ('BMW', 0.7272197604179382), ('Maserati', 0.7194912433624268), ('VW', 0.6961471438407898)]


In [73]:
c = g_news.similarity('Italy', 'Rome')
print(c)

0.5555642


In [1]:
import re
import spacy
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import pandas as pd
from nltk.stem.porter import *
import nltk

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv('dataset/movies.csv', index_col=0)

In [5]:
movies_sample = df.sample(frac = 0.2, replace=False)

In [8]:
movies_sample = movies_sample.reset_index(drop=True)

nlp = spacy.load('en_core_web_sm')

In [9]:
sw_english = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [10]:
def preprocess(string):
    ###
    # Deixa apenas elementos alfanuméricos
    string = re.sub(r"[^a-zA-Z0-9]+", ' ', string)
    ###
    # deixa todas as palavras minúsculas
    string = string.lower()
    ###
    words = word_tokenize(string)
    ###
    # Remove Stopwords
    filtered_words = [word for word in words if word not in sw_english]
    # Aplica o Stemming
    stem_words = []
    for w in filtered_words:
        s_words = stemmer.stem(w)
        stem_words.append(s_words)
    ###
    # Retorna a lista de palavras pré-processadas
    return stem_words

In [11]:
movies_sample["filtered_words"] = movies_sample['text'].apply(lambda x: preprocess(x))

In [12]:
words_list = movies_sample['filtered_words'].to_list()

In [13]:
from gensim.models.phrases import Phraser, Phrases, ENGLISH_CONNECTOR_WORDS

In [14]:
# Lista de listas (list comp.)
# n-gram - unigram = love, trigrama = because_of_you

phrases = Phrases(words_list, min_count=1, connector_words=ENGLISH_CONNECTOR_WORDS, threshold=1)

bigram = Phraser(phrases)

sentences = list(bigram[words_list])

In [15]:
print(sentences[3705])

['pokemon', '3', 'littl', 'three_four', 'episod_tv', 'seri', 'strung_togeth', 'without', 'usual', 'commerci', 'stori_typic', 'pokemon', 'conflict', 'fight', 'resolut', 'happi_end', 'noth_origin', 'unusu_anim', 'hole_plot', 'fill', 'close_credit', 'without', 'explan_everyth', 'bit', 'sweet', 'br_br', 'see', 'big_screen', 'reason', 'part', 'child', 'world', 'son', 'enjoy', 'pokemon', 'show_interest', 'like', 'closer', 'see_film', 'theatr', 'still', 'differ', 'see', 'tube', 'son', 'enjoy_full', 'movi_go', 'experi', 'gave_movi', '4_mostli', 'children', 'point_view', 'br_br']


In [16]:
model = Word2Vec(sentences, min_count=2, workers=2)

In [17]:
model.wv.most_similar('bad')

[('good', 0.9993441700935364),
 ('one', 0.9987869262695312),
 ('film', 0.9985126852989197),
 ('watch', 0.9982575178146362),
 ('know', 0.9981472492218018),
 ('movi', 0.9980112314224243),
 ('end', 0.9979390501976013),
 ('think', 0.9976657629013062),
 ('stori', 0.9976332783699036),
 ('plot', 0.9975210428237915)]

#### Salvando o modelo

In [18]:
model.save('model_w2v.bin')

In [19]:
model2 = Word2Vec.load('model_w2v.bin')